# Comparing Pitchfork reviews and artist lyrics

In [179]:
import pitchfork.pitchfork as pitchfork
from pitchfork.pitchfork.pitchfork import Review # For restoring saved reviews
from bs4 import BeautifulSoup
import json
import pickle
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import time

## Load the lyrics data

In [7]:
# Load the lyrics
artists_rap = json.load(open('lyrics/rap_lyrics.json'))
artists_rock = json.load(open('lyrics/rock_lyrics.json'))
artists_country = json.load(open('lyrics/country_lyrics.json'))
artists_raw = artists_rap + artists_rock + artists_country

In [10]:
# Determine
def getArtistImage(songs):    
    urls = [None]
    for song in songs:
        if song['raw']['album'] is not None:
            try:                
                urls.append(song['raw']['album']['artist']['image_url'])                
            except:
                pass
    
    return Counter(urls).most_common()[0][0] # Most common image

# Use a generator to exclude artists with too few songs
def filterBySongCount(artists, min_count=10, max_count=10000):
    for artist in artists:        
        num_songs = len(artist['songs'])
        if (num_songs >= min_count) and (num_songs < max_count): yield artist

min_songs, max_songs = 10, 2500
artists = list(filterBySongCount(artists_raw, min_songs, max_songs))

# Artist images
for artist in artists:
    artist['image_url'] = getArtistImage(artist['songs'])
    
    # Kludge
    if artist['artist'] == 'Black Thought':
        artist['image_url'] = 'https://images.genius.com/23de511f4ac50d900128ea363b7d81b3.450x320x1.jpg'
    if artist['artist'] == 'André 3000':
        artist['image_url'] = 'http://images.genius.com/64b15c9489c65f5bf8f6577334347404.434x434x1.jpg'            
        
# Gender
labels_gender = np.array([a['gender'] for a in artists])
mask_female = np.array([g=='female' for g in labels_gender])
mask_male   = np.array([g=='male'   for g in labels_gender])
print("You have lyrics from {} artists, with a minimum of {} songs per artist.\nFemale: {}, male: {}"
      .format(len(artists), min_songs, (labels_gender=='female').sum(), (labels_gender=='male').sum()))

# Add all song lyrics to a single list
all_songs = []
for artist, gender in zip(artists, labels_gender):
    for song in artist['songs']:        
        song['gender'] = gender
        all_songs.append(song)
              
song_count = len(all_songs)
print("Database contains {} songs.".format(song_count))

# Store all lyrics in a single string
all_lyrics = " ".join([song['lyrics'] for song in all_songs])

You have lyrics from 303 artists, with a minimum of 10 songs per artist.
Female: 79, male: 222
Database contains 58239 songs.


## Get list of all artist names and their albums

In [198]:
# For each artist, get a list of their albums
artist_albums = {}
min_songs = 7
for artist in artists:
    album_song_counts = {} # Keep track of how many songs show up for each album
    for song in artist['songs']:        
        album = song['album']        
        if album in album_song_counts:
            album_song_counts[album] += 1
        elif album is not None:
            album_song_counts[album]  = 1

    # Only store albums with a bare minimum of songs (a rough indication of a real album)    
    artist_albums[artist['artist']] = [album[0] for album in album_song_counts.items() if album[1] >= min_songs]

# Filter out artists without any albums matching the criteria    
artist_albums = {artist: albums for artist, albums in artist_albums.items() if len(albums)>0}

# Report
print("Found {} artists with albums containing at least {} songs.".format(len(artist_albums), min_songs))

Found 297 artists with albums containing at least 7 songs.


## Search Pitchfork for the album names we've collected

In [228]:
# Check if the pickle file already exists
filename_album_reviews = "album_reviews.pickle"
filename_missing_reviews = "missing_reviews.pickle"
if os.path.exists(filename_album_reviews) and os.path.getsize(filename_album_reviews) > 0:
    artist_reviews = pickle.load(open(filename_album_reviews, 'rb'))    
else:
    artist_reviews = {}
if os.path.exists(filename_missing_reviews) and os.path.getsize(filename_missing_reviews) > 0:
    missing_reviews = pickle.load(open(filename_missing_reviews, 'rb'))
else:
    missing_reviews = {}
    
# Search for reviews for each artist's albums    
start = time.time()
for artist in artist_albums:
    print("\n{row}\nSearching for reviews of albums by '{artist}'...".format(row="-"*50, artist=artist))
    try:
        album_reviews = artist_reviews[artist] if artist in artist_reviews else {}
        no_reviews = missing_reviews[artist] if artist in missing_reviews else []
        for album in artist_albums[artist]:
            if album not in album_reviews and album not in no_reviews:
                try:
                    review = pitchfork.search(artist, album)
                    album_reviews[album] = review.to_json()
                    print('Found a review for "{album}" / ({rev_name}).'.format(album=album, rev_name=review.album()))
                except:
                    no_reviews.append(album)
                    print("Could not find a review for {album}".format(album=album))
                    
                time.sleep(10) # Wait before pulling a new review
            else:
                print("Already have the review for {}, or we already know it's not reviewed.".format(album))

        # If reviews were found, add them to the dictionary
        if len(album_reviews) > 0:
            artist_reviews[artist] = album_reviews
        if len(no_reviews) > 0:
            missing_reviews[artist] = no_reviews # Keep track of missing reviews

        # Save all reviews for current artist into the pickle
        pickle.dump(artist_reviews, open(filename_album_reviews, 'wb'))
        pickle.dump(missing_reviews, open(filename_missing_reviews, 'wb'))
    except Exception as e:
        print("Error:\n{e}".format(e=e))
        time.sleep(5*60) # Wait five minutes
    
end = time.time()
print("\n\n{row}\nDone scraping Pitchfork reviews!\nTime elapsed: {t} hours".format(row="*"*60, t=(start-end)/60/60))


--------------------------------------------------
Searching for reviews of albums by '2Pac'...
Already have the review for R U Still Down? (Remember Me), or we already know it's not reviewed.
Already have the review for The Rose That Grew From Concrete (Book), or we already know it's not reviewed.
Already have the review for All Eyez on Me, or we already know it's not reviewed.
Already have the review for 2Pac Unreleased, or we already know it's not reviewed.
Already have the review for Strictly 4 My N.I.G.G.A.Z..., or we already know it's not reviewed.
Already have the review for Beginnings: The Lost Tapes 1988–1991, or we already know it's not reviewed.
Already have the review for The Don Killuminati: The 7 Day Theory, or we already know it's not reviewed.
Already have the review for Better Dayz, or we already know it's not reviewed.
Already have the review for Until the End of Time, or we already know it's not reviewed.
Already have the review for The Rose, Vol. 2: Music Inspired 


--------------------------------------------------
Searching for reviews of albums by 'Dr. Dre'...
Already have the review for 2001, or we already know it's not reviewed.
Already have the review for Compton, or we already know it's not reviewed.
Already have the review for The Chronic, or we already know it's not reviewed.

--------------------------------------------------
Searching for reviews of albums by 'Kendrick Lamar'...
Already have the review for Section.80, or we already know it's not reviewed.
Already have the review for Overly Dedicated, or we already know it's not reviewed.
Already have the review for To Pimp a Butterfly, or we already know it's not reviewed.
Already have the review for C4, or we already know it's not reviewed.
Already have the review for Training Day, or we already know it's not reviewed.
Already have the review for good kid, m.A.A.d city, or we already know it's not reviewed.
Already have the review for Compton State of Mind, or we already know it's not


--------------------------------------------------
Searching for reviews of albums by 'Method Man'...
Already have the review for The Meth Lab, or we already know it's not reviewed.
Already have the review for 4:21... The Day After, or we already know it's not reviewed.
Already have the review for Tical 0: The Prequel, or we already know it's not reviewed.
Already have the review for Tical, or we already know it's not reviewed.
Already have the review for Tical 2000: Judgement Day, or we already know it's not reviewed.

--------------------------------------------------
Searching for reviews of albums by 'Yasiin Bey'...
Already have the review for The Ecstatic, or we already know it's not reviewed.
Already have the review for The New Danger, or we already know it's not reviewed.
Already have the review for Black on Both Sides, or we already know it's not reviewed.
Already have the review for True Magic, or we already know it's not reviewed.

-------------------------------------------


--------------------------------------------------
Searching for reviews of albums by 'Big L'...
Already have the review for Harlem's Finest - A Freestyle History, or we already know it's not reviewed.
Already have the review for Lifestylez Ov Da Poor & Dangerous, or we already know it's not reviewed.
Already have the review for The Danger Zone, or we already know it's not reviewed.
Already have the review for The Big Picture, or we already know it's not reviewed.
Already have the review for The Archives 1996-2000 (Mixtape), or we already know it's not reviewed.
Already have the review for Return of the Devil’s Son, or we already know it's not reviewed.

--------------------------------------------------
Searching for reviews of albums by 'Ghostface Killah'...
Already have the review for Ironman, or we already know it's not reviewed.
Already have the review for Apollo Kids, or we already know it's not reviewed.
Already have the review for 36 Seasons, or we already know it's not review


--------------------------------------------------
Searching for reviews of albums by 'Raekwon'...
Already have the review for Immobilarity, or we already know it's not reviewed.
Already have the review for Only Built 4 Cuban Linx... Pt. II, or we already know it's not reviewed.
Already have the review for Fly International Luxurious Art, or we already know it's not reviewed.
Already have the review for The Vatican Mixtape, Vol. 4, or we already know it's not reviewed.
Already have the review for Lost Jewlry, or we already know it's not reviewed.
Already have the review for #ThrowBackThursday, or we already know it's not reviewed.
Already have the review for The Lex Diamond Story, or we already know it's not reviewed.
Already have the review for Unexpected Victory, or we already know it's not reviewed.
Already have the review for House of Wax: The Vatican Mixtape, Vol. 3, or we already know it's not reviewed.
Already have the review for The Vatican Mixtape, Vol. 1, or we already know 

...


--------------------------------------------------
Searching for reviews of albums by 'Sheryl Crow'...
Already have the review for 100 Miles From Memphis, or we already know it's not reviewed.
Already have the review for C'mon, C'mon, or we already know it's not reviewed.
Already have the review for Sheryl Crow, or we already know it's not reviewed.
Already have the review for Tuesday Night Music Club (Deluxe Edition), or we already know it's not reviewed.
Already have the review for Tuesday Night Music Club, or we already know it's not reviewed.
Already have the review for , or we already know it's not reviewed.
Already have the review for Home for Christmas, or we already know it's not reviewed.
Already have the review for Be Myself, or we already know it's not reviewed.
Already have the review for Wildflower, or we already know it's not reviewed.
Already have the review for The Globe Sessions, or we already know it's not reviewed.
Already have the review for Detours, or we already 


--------------------------------------------------
Searching for reviews of albums by 'Wanda Jackson'...
Already have the review for Rockin' With Wanda, or we already know it's not reviewed.
Already have the review for , or we already know it's not reviewed.
Already have the review for I Remember Elvis, or we already know it's not reviewed.
Already have the review for Unfinished Business, or we already know it's not reviewed.
Already have the review for Reckless Love Affair, or we already know it's not reviewed.
Already have the review for The Party Ain't Over, or we already know it's not reviewed.
Already have the review for Let's Have a Party, or we already know it's not reviewed.
Already have the review for There's a Party Goin' On, or we already know it's not reviewed.
Already have the review for Heart Trouble, or we already know it's not reviewed.
Already have the review for Love Me Forever, or we already know it's not reviewed.
Already have the review for Tears At The Grand Ole 


--------------------------------------------------
Searching for reviews of albums by 'Hank Williams'...
Already have the review for , or we already know it's not reviewed.
Already have the review for I Saw The Light, or we already know it's not reviewed.
Already have the review for Hank Williams Sings, or we already know it's not reviewed.
Already have the review for Honky Tonkin' , or we already know it's not reviewed.
Already have the review for Hank Williams as Luke the Drifter, or we already know it's not reviewed.
Already have the review for The Unreleased Recordings, or we already know it's not reviewed.
Already have the review for Memorial Album, or we already know it's not reviewed.
Already have the review for The Lonesome Sound of Hank Williams, or we already know it's not reviewed.
Already have the review for The Unforgettable Hank Williams, or we already know it's not reviewed.
Already have the review for Moanin' The Blues, or we already know it's not reviewed.
Already hav

Could not find a review for T-R-O-U-B-L-E
Could not find a review for A Travis Tritt Christmas: A Loving Time of Year
Could not find a review for It's All About to Change
Could not find a review for The Restless Kind
Could not find a review for Down the Road I Go
Could not find a review for Ten Feet Tall and Bulletproof
Could not find a review for 
Could not find a review for Strong Enough
Could not find a review for My Honky Tonk History
Could not find a review for Country Club
Could not find a review for The Storm
Could not find a review for No More Looking Over My Shoulder

--------------------------------------------------
Searching for reviews of albums by 'Eric Church'...
Could not find a review for  Carolina
Could not find a review for The Outsiders
Could not find a review for Sinners Like Me
Could not find a review for Mr. Misunderstood
Could not find a review for Chief
Could not find a review for 

--------------------------------------------------
Searching for reviews of alb

Could not find a review for Christmas
Could not find a review for Moonshine in the Trunk
Could not find a review for Wheelhouse
Could not find a review for Mud On The Tires
Could not find a review for Time Well Wasted
Could not find a review for 
Could not find a review for 5th Gear
Could not find a review for Part II
Could not find a review for This Is Country Music
Could not find a review for American Saturday Night
Could not find a review for Who Needs Pictures
Could not find a review for Love and War

--------------------------------------------------
Searching for reviews of albums by 'Kip Moore'...
Could not find a review for Wild Ones (Deluxe)
Could not find a review for Up All Night
Could not find a review for Slowheart
Could not find a review for 

--------------------------------------------------
Searching for reviews of albums by 'Brett Eldredge'...
Could not find a review for Glow
Could not find a review for Bring You Back
Could not find a review for Brett Eldredge
Could n

In [207]:
artist_reviews.keys()

dict_keys(['Kanye West', 'The Beatles', '2Pac', 'Eminem', 'The Notorious B.I.G.', 'Nas', 'Ice Cube', 'JAY-Z', 'Snoop Dogg', 'Dr. Dre', 'Kendrick Lamar', 'A$AP Rocky', '50 Cent', 'Busta Rhymes'])

In [208]:
artist_reviews['Kendrick Lamar'].keys()

dict_keys(['Section.80', 'To Pimp a Butterfly', 'good kid, m.A.A.d city', 'DAMN.', 'untitled unmastered.'])